# MENACE: Machine Educable Noughts And Crosses Engine

## Introduction

MENACE (Machine Educable Noughts And Crosses Engine) is a mechanical learning machine designed by Donald Michie in 1961. It was created to demonstrate the concept of reinforcement learning, long before modern machine learning techniques were developed.

## Historical Context

Donald Michie (1923-2007) was a British researcher and a pioneer in artificial intelligence. He worked at Bletchley Park during World War II, contributing to the effort to break the German Enigma code alongside Alan Turing. After the war, Michie pursued a career in biology before transitioning to artificial intelligence research.

MENACE was conceived during a time when computers were still in their infancy and not widely accessible. Michie's invention demonstrated that learning algorithms could be implemented without complex electronic computers, using simple mechanical means.

## How MENACE Works

MENACE uses a collection of matchboxes to "learn" how to play Tic-Tac-Toe (also known as Noughts and Crosses). Here's a brief overview of its operation:

1. **Representation**: Each possible game state is represented by a matchbox.
2. **Decision Making**: Inside each matchbox are colored beads, each color corresponding to a possible move.
3. **Learning**: 
   - To make a move, MENACE randomly selects a bead from the appropriate matchbox.
   - After the game, beads are added or removed based on the outcome:
     - Win: Add three beads of the colors used
     - Draw: Add one bead of the colors used
     - Loss: Remove one bead of the colors used

Over time, MENACE learns to make better moves by adjusting the probabilities of selecting each move in different game states.

## Significance

MENACE is significant for several reasons:

1. **Early Reinforcement Learning**: It demonstrated a simple yet effective form of reinforcement learning, a concept that would become crucial in modern AI and machine learning.
2. **Mechanical Computation**: MENACE showed that learning algorithms could be implemented without electronic computers, emphasizing the underlying principles of machine learning.
3. **Interpretability**: The physical nature of MENACE made it easy to understand and visualize the learning process, a quality often lacking in modern, more complex machine learning models.
4. **Inspiration**: MENACE has inspired numerous recreations and adaptations, including digital implementations and educational tools.

## Modern Relevance

While MENACE itself is no longer used for practical applications, the principles it demonstrates remain relevant:

1. **Reinforcement Learning**: The core idea behind MENACE is still fundamental to modern reinforcement learning algorithms used in various fields, from game playing to robotics.
2. **Explainable AI**: As AI systems become more complex, there's a growing interest in making them more interpretable. MENACE's transparent decision-making process resonates with current efforts in explainable AI.
3. **Educational Tool**: MENACE continues to be an excellent educational tool for introducing concepts of machine learning and reinforcement learning in an accessible, hands-on manner.

## Conclusion

MENACE stands as a testament to the ingenuity of early AI researchers and the timeless nature of fundamental machine learning concepts. Its simplicity, elegance, and effectiveness continue to inspire and educate, bridging the gap between the early days of AI and the cutting-edge technologies of today.

In [ ]:
import random

import matplotlib.pyplot as matplotlib_pyplot
import numpy as numpy

In [ ]:
# Each MENACE mathbox has a state of the board 0/1 for each position with 10 initial beads.
# Matchboxes is a hash of board states (0-8) and a tensor of the number of beads. 
# A move requires the state of the board.
class MENACE:
    def __init__(self, initial_beads=10):
        self.matchboxes = {}
        self.initial_beads = initial_beads
        self.moves = []
        self.first_box_beads_history = []

    def get_state(self, board):
        return ','.join(map(str, board))
    
    def reset_history(self):
        self.first_box_beads_history = []

    def get_move(self, board):
        state = self.get_state(board)
        if state not in self.matchboxes:
            self.matchboxes[state] = [self.initial_beads if cell == 0 else 0 for cell in board]

        # Record a history of the first move of the first matchbox.
        if state == '0,0,0,0,0,0,0,0,0':
            self.first_box_beads_history.append(self.matchboxes[state][:])

        # If it runs out of beads - just pick a random move.
        total_beads = sum(self.matchboxes[state])
        if total_beads == 0:
            return random.choice([i for i, beads in enumerate(self.matchboxes[state]) if beads == 0])
        else:
            # Select a move 0-8, using the number of beads left to weight the probability.
            move = random.choices(range(9), weights=self.matchboxes[state])[0]
            self.moves.append((state, move))
            return move

    # Set reward - min value 0 beads.
    def update(self, reward):
        for state, move in self.moves:
            self.matchboxes[state][move] += reward
            self.matchboxes[state][move] = max(self.matchboxes[state][move], 0)
        self.moves = []

In [ ]:
# From a board state, make a random move.
# Random moves will almost create a perfectly valid uptick in wins.
class RandomPlayer:
    def __init__(self):
        self.moves = []

    def get_move(self, board):
        valid_moves = [i for i, cell in enumerate(board) if cell == 0]
        move = random.choice(valid_moves)
        self.moves.append((self.get_state(board), move))
        return move

    # Random player doesn't learn, so we just clear the moves
    def update(self, reward):
        self.moves = []

    def get_state(self, board):
        return ','.join(map(str, board))

In [ ]:
# A board has 0-8 spaces in a tic-tac-toe board - winning is rows, columns or diagonals.
class Board:
    def __init__(self):
        self.states = [0] * 9

    def __getitem__(self, index):
        return self.states[index]

    def __setitem__(self, index, value):
        self.states[index] = value

    def __iter__(self):
        return iter(self.states)

    def __len__(self):
        return len(self.states)        
    
    # From a board return True if there are 3 by rows, columns or diagonals.
    def check_win(self):
        lines = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6]              # Diagonals
        ]
        for line in lines:
            if self.states[line[0]] != 0 and self.states[line[0]] == self.states[line[1]] == self.states[line[2]]:
                return True
        return False

In [ ]:
# Create empty board, MENACE always goes first.
# Alternately, get a MENACE move, get a player move.
# Check for a win - return 1 if MENACE one -1 if player has one.
def play_game(menace, player):
    board = Board()
    menace_turn = True

    while True:
        if menace_turn:
            move = menace.get_move(board)
            board[move] = 1
        else:
            move = player.get_move(board)
            board[move] = -1

        if board.check_win():
            return 1 if menace_turn else -1
        if all(cell != 0 for cell in board):
            return 0

        menace_turn = not menace_turn

In [169]:
# Initialise
menace = MENACE()
player = RandomPlayer()
results = []
num_games = 200

In [168]:
# If the result is a win (1) - add 3 beads.
# If the result is a draw (0) - add 1 bead.
# If the result is a loss (-1) - remove 1 bead.
for game in range(num_games):
    result = play_game(menace, player)
    if result == 1:
        menace.update(3)
    elif result == 0:
        menace.update(1)
    else:
        menace.update(-1)
    results.append(result)

In [ ]:
# Assuming 'results' is a list of game outcomes (1 for win, 0 for draw, -1 for loss)
# and 'menace' is your MENACE instance
cumulative_results = numpy.cumsum(results)
games_played = numpy.arange(1, len(results) + 1)
running_average = cumulative_results / games_played

fig, (ax1, ax2, ax3) = matplotlib_pyplot.subplots(3, 1, figsize=(12, 16))

# Plot performance
ax1.plot(games_played, running_average)
ax1.set_title(f"MENACE Performance Over {len(results)} Games")
ax1.set_xlabel("Games Played")
ax1.set_ylabel("Running Average Score")
ax1.grid(True)

# Plot Matchbox 0
first_box_beads_history = numpy.array(menace.first_box_beads_history).T
for i in range(9):
    ax2.plot(range(len(first_box_beads_history[i])), first_box_beads_history[i], label=f'Position {i}')

ax2.set_title("Beads in First Matchbox for Each Position Over Time")
ax2.set_xlabel("Game Number")
ax2.set_ylabel("Number of Beads")
ax2.legend()
ax2.grid(True)

# Plot against tic-tac-toe
final_beads = menace.matchboxes['0,0,0,0,0,0,0,0,0']
beads_array = numpy.array(final_beads).reshape(3, 3)

ax3.imshow(beads_array, cmap='YlOrRd')

for i in range(3):
    for j in range(3):
        ax3.text(j, i, f'{beads_array[i, j]:.1f}', ha='center', va='center', color='black', fontweight='bold')

ax3.set_xticks([])
ax3.set_yticks([])
ax3.set_title('Number of Beads for First Move')

for i in range(4):
    ax3.axhline(i - 0.5, color='black', linewidth=2)
    ax3.axvline(i - 0.5, color='black', linewidth=2)

matplotlib_pyplot.tight_layout()

win_rate = results.count(1) / len(results)
draw_rate = results.count(0) / len(results)
loss_rate = results.count(-1) / len(results)

print(f"Win rate: {win_rate:.2%}")
print(f"Draw rate: {draw_rate:.2%}")
print(f"Loss rate: {loss_rate:.2%}")